<a href="https://colab.research.google.com/github/fraco03/6D_pose/blob/pose_rgb/notebooks/pose_rgb/pose_rgb_pointnet_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

# Clone or pull part
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/content/6D_pose"
branch = "pose_rgb"

if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url}")

if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

%cd 6D_pose

Cloning into '6D_pose'...
remote: Enumerating objects: 349, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 349 (delta 1), reused 4 (delta 1), pack-reused 334 (from 1)
Receiving objects: 100% (349/349), 5.57 MiB | 29.39 MiB/s, done.
Resolving deltas: 100% (167/167), done.
Cloned https://github.com/fraco03/6D_pose.git
/content/6D_pose


In [2]:
from google.colab import drive
from utils.load_data import mount_drive

mount_drive()

dataset_root = "/content/drive/MyDrive/Linemod_preprocessed"
print(f"\n✅ Setup complete!")
print(f"📁 Dataset path: {dataset_root}")

Mounted at /content/drive
✅ Drive mounted at /content/drive

✅ Setup complete!
📁 Dataset path: /content/drive/MyDrive/Linemod_preprocessed


In [3]:
!pip install plyfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.9 MB/s eta 0:00:00


In [4]:
from src.pose_rgb.pointcloud_dataset import LineModPointCloudDataset
from src.pose_rgb.pointnet_model import PointNetPose
from src.pose_rgb.loss import AutomaticWeightedLoss
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🔥 Using device: {DEVICE}")

🔥 Using device: cuda


## 📝 Note sulla Loss Function

**Differenza tra PointNet e RGB approach:**

- **RGB (ResNet + TranslationNet)**:
  - Predice `[dx, dy, log(z)]` → poi pinhole projection → `[X, Y, Z]`
  - Usa `DisentangledTranslationLoss`: separa XY da Z
  - Ha senso perché XY dipendono dalla geometria pinhole, Z è indipendente

- **PointNet**:
  - Predice **direttamente** `[X, Y, Z]` dalla point cloud
  - Usa loss **unificata** per translation: tratta X, Y, Z simmetricamente
  - Non c'è pinhole projection, quindi non ha senso separare XY da Z

## ⚡ Performance Optimizations

**Ottimizzazioni applicate per velocizzare il training:**

1. **Riduzione punti**: 1024 → 512 punti per point cloud (~2x speed-up)
2. **Cached YAML files**: `linemod_config` cacha `info.yml` e `gt.yml` invece di aprirli ogni volta (~10-20x speed-up!)
   - **Critico su Google Drive**: I/O latency è molto alta, caching essenziale
3. **Torch sampling**: `torch.randperm` invece di `np.random.choice` (~1.5x speed-up)
4. **Mixed precision**: FP16/FP32 automatico con `torch.cuda.amp` (~1.5x speed-up)
5. **Batch size**: Aumentato da 32 → 64 per migliore GPU utilization
6. **DataLoader**: `num_workers=4` + `pin_memory=True` per I/O parallelo

**Speed-up totale stimato: ~20-30x** rispetto alla versione iniziale! 🚀

**Nota**: Il primo batch può richiedere più tempo per caricare e cachare tutti i YAML files, poi diventa molto più veloce.

In [5]:
# Crea dataset con point clouds
# use_rgb=True -> point cloud con 6 canali [x,y,z,r,g,b]
# use_rgb=False -> point cloud con 3 canali [x,y,z]

train_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='train',
    num_points=512,  # Ridotto per speed-up (era 1024)
    use_rgb=True      # Include RGB
)

test_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='test',
    num_points=512,
    use_rgb=True
)

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

✅ LineModConfig initialized: /content/drive/.shortcut-targets-by-id/1sm0RTmd3Q00Uw99X5cErD8SxrnPrGS6v/Linemod_preprocessed
📊 Loaded 2373 samples for train split
📊 Loaded 13407 samples for test split
Train samples: 2373
Test samples: 13407


In [6]:
# Visualizza un sample
sample = train_dataset[0]

print("Sample keys:", sample.keys())
print(f"Point cloud shape: {sample['point_cloud'].shape}")  # (1024, 6)
print(f"Rotation shape: {sample['rotation'].shape}")        # (4,)
print(f"Translation shape: {sample['translation'].shape}")  # (3,)
print(f"\nRotation (quat): {sample['rotation']}")
print(f"Translation (m): {sample['translation']}")

Sample keys: dict_keys(['point_cloud', 'rotation', 'translation', 'object_id', 'img_id', 'cam_K', 'bbox'])
Point cloud shape: torch.Size([512, 6])
Rotation shape: (4,)
Translation shape: torch.Size([3])

Rotation (quat): [ 0.33261785  0.64730227  0.6364495  -0.2555329 ]
Translation (m): tensor([-0.1036, -0.0498,  1.0251])


In [7]:
# DataLoaders - Ottimizzati per performance
train_loader = DataLoader(
    train_dataset,
    batch_size=64,  # Aumentato da 32 per migliore GPU utilization
    shuffle=True,
    num_workers=2,  # Aumentato da 2
    pin_memory=True  # Velocizza transfer CPU->GPU
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [8]:
import os
import json
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt
from torch.amp import autocast, GradScaler

# ==========================================
# HYPERPARAMETERS
# ==========================================
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50
USE_MIXED_PRECISION = True  # Mixed precision for speed-up

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
CHECKPOINT_DIR = f'/content/drive/MyDrive/runs/pointnet_{timestamp}'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Initialize PointNet Model
# input_channels=6 because we use [x,y,z,r,g,b]
model = PointNetPose(input_channels=6, use_batch_norm=True).to(DEVICE)

# Loss for PointNet
criterion = AutomaticWeightedLoss(use_disentangled=False).to(DEVICE)

# Optimizer
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(criterion.parameters()),
    lr=LEARNING_RATE
)

# Mixed Precision Scaler
scaler = GradScaler('cuda') if USE_MIXED_PRECISION else None

train_losses = []
val_losses = []
best_val_loss = float('inf')
best_epoch = 0

print(f"\n🔥 STARTING POINTNET TRAINING on {DEVICE}...")
print(f"📁 Checkpoints: {CHECKPOINT_DIR}")
print(f"⚙️  Loss mode: Unified Translation (no XY/Z separation)")
print(f"⚡ Mixed Precision: {USE_MIXED_PRECISION}")


🔥 STARTING POINTNET TRAINING on cuda...
📁 Checkpoints: /content/drive/MyDrive/runs/pointnet_20251217_132025
⚙️  Loss mode: Unified Translation (no XY/Z separation)
⚡ Mixed Precision: True


In [9]:
# ==========================================
# TRAINING LOOP (con Mixed Precision)
# ==========================================
for epoch in range(NUM_EPOCHS):

    # --- TRAIN PHASE ---
    model.train()
    running_train_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")

    for batch in pbar:
        # Move to device
        point_clouds = batch['point_cloud'].to(DEVICE, non_blocking=True)  # (B, N, 6)
        gt_rot = batch['rotation'].to(DEVICE, non_blocking=True)           # (B, 4)
        gt_trans = batch['translation'].to(DEVICE, non_blocking=True)      # (B, 3) in meters

        optimizer.zero_grad()

        # Mixed Precision Forward + Backward
        if USE_MIXED_PRECISION:
            with autocast('cuda'):
                pred_rot, pred_trans = model(point_clouds)
                loss, l_r, l_t, _ = criterion(pred_rot, gt_rot, pred_trans, gt_trans)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            pred_rot, pred_trans = model(point_clouds)
            loss, l_r, l_t, _ = criterion(pred_rot, gt_rot, pred_trans, gt_trans)
            loss.backward()
            optimizer.step()

        running_train_loss += loss.item()

        pbar.set_postfix({
            'L_Tot': f"{loss.item():.2f}",
            'L_Rot': f"{l_r.item():.2f}",
            'L_Trans': f"{l_t.item():.3f}"
        })

    avg_train_loss = running_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # --- VALIDATION PHASE ---
    model.eval()
    running_val_loss = 0.0
    val_batches_limit = 50
    count_batches = 0

    with torch.no_grad():
        val_iterator = islice(test_loader, val_batches_limit)
        val_pbar = tqdm(val_iterator, total=val_batches_limit, desc="Validating")

        for batch in val_pbar:
            point_clouds = batch['point_cloud'].to(DEVICE, non_blocking=True)
            gt_rot = batch['rotation'].to(DEVICE, non_blocking=True)
            gt_trans = batch['translation'].to(DEVICE, non_blocking=True)

            if USE_MIXED_PRECISION:
                with autocast('cuda'):
                    pred_rot, pred_trans = model(point_clouds)
                    loss, _, _, _ = criterion(pred_rot, gt_rot, pred_trans, gt_trans)
            else:
                pred_rot, pred_trans = model(point_clouds)
                loss, _, _, _ = criterion(pred_rot, gt_rot, pred_trans, gt_trans)

            running_val_loss += loss.item()
            count_batches += 1

    avg_val_loss = running_val_loss / count_batches if count_batches > 0 else 0.0
    val_losses.append(avg_val_loss)

    # --- REPORT & SAVE ---
    print(f"📊 Epoch {epoch+1}: Train={avg_train_loss:.4f} | Val={avg_val_loss:.4f}")

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_epoch = epoch + 1

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'criterion_state_dict': criterion.state_dict(),
            'val_loss': best_val_loss
        }, os.path.join(CHECKPOINT_DIR, "best_model.pth"))

        print(f"🏆 New Best Model! (Loss: {best_val_loss:.4f})")

    # Save last checkpoint
    if (epoch + 1) == NUM_EPOCHS:
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'criterion_state_dict': criterion.state_dict(),
        }, os.path.join(CHECKPOINT_DIR, f"checkpoint_ep{epoch+1}.pth"))

print("\n🎉 TRAINING COMPLETE!")

Validating: 100%|██████████| 50/50 [24:19<00:00, 29.19s/it]


📊 Epoch 1: Train=0.8885 | Val=0.2928
🏆 New Best Model! (Loss: 0.2928)


Validating: 100%|██████████| 50/50 [01:57<00:00,  2.34s/it]


📊 Epoch 2: Train=0.4411 | Val=0.2863
🏆 New Best Model! (Loss: 0.2863)


Validating: 100%|██████████| 50/50 [01:49<00:00,  2.20s/it]


...

📊 Epoch 25: Train=0.1160 | Val=0.0624


Validating: 100%|██████████| 50/50 [01:50<00:00,  2.22s/it]


📊 Epoch 26: Train=0.1065 | Val=0.0333


Epoch 27/50 [Train]:   5%|▌         | 2/38 [01:01<18:28, 30.79s/it, L_Tot=0.07, L_Rot=0.24, L_Trans=0.006]


KeyboardInterrupt: 

In [ ]:
# Plot training history
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', marker='o', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', marker='s', alpha=0.7)
if best_epoch > 0:
    plt.axvline(x=best_epoch-1, color='r', linestyle='--', alpha=0.5,
                label=f'Best Epoch ({best_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('PointNet Training History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Train Loss', marker='o', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', marker='s', alpha=0.7)
if best_epoch > 0:
    plt.axvline(x=best_epoch-1, color='r', linestyle='--', alpha=0.5,
                label=f'Best Epoch ({best_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.yscale('log')
plt.title('PointNet Training History (Log)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CHECKPOINT_DIR, 'training_history.png'), dpi=150)
plt.show()

print(f"\n📊 Training Statistics:")
print(f"   Best epoch: {best_epoch}")
print(f"   Best val loss: {best_val_loss:.6f}")
print(f"   Final train loss: {train_losses[-1]:.6f}")

# Save history
history = {
    'train_losses': [float(x) for x in train_losses],
    'val_losses': [float(x) for x in val_losses],
    'best_epoch': int(best_epoch),
    'best_val_loss': float(best_val_loss),
    'timestamp': timestamp
}

with open(os.path.join(CHECKPOINT_DIR, 'history.json'), 'w') as f:
    json.dump(history, f, indent=2)

## Visualize Point Cloud Sample

Visualizing point cloud from random point in the dataset

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Choose sample
sample = train_dataset[100]
pc = sample['point_cloud'].numpy()  # (512, 6) [x, y, z, r, g, b]

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot 3D points with RGB colors
ax.scatter(pc[:, 0], pc[:, 1], pc[:, 2],
           c=pc[:, 3:6], s=2, alpha=0.6)

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
ax.set_title(f'Point Cloud - Object {sample["object_id"]}')
plt.show()

print(f"Object ID: {sample['object_id']}")
print(f"Rotation (quat): {sample['rotation']}")
print(f"Translation (m): {sample['translation']}")